In [1]:
import pandas as pd
import os
from datetime import timezone
import datetime
import time
import math
import copy
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine

In [3]:
MasterData = pd.read_csv('Master_Data_File.csv', index_col='index')
MasterData

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
index,,,,,,,,
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
...,...,...,...,...,...,...,...,...
3168102,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190
3168103,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391
3168104,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527


In [138]:
last_date=0
dupliate_indices=[]
for i in range(len(MasterData)) :
    unix = MasterData.loc[i,'Unix Timestamp']
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            dupliate_indices.append(i)
            print('Difference: ', difference, ' index ',i)
    last_date=unix

Difference:  0  index  1510679
Difference:  0  index  1516283
Difference:  0  index  1632820
Difference:  0  index  1746185
Difference:  0  index  1759685
Difference:  0  index  1764008
Difference:  0  index  1781287
Difference:  0  index  1785610
Difference:  0  index  1814411
Difference:  0  index  1824492
Difference:  0  index  1841774
Difference:  0  index  1844654
Difference:  0  index  1864816
Difference:  0  index  1884977
Difference:  0  index  1915218
Difference:  0  index  1933939
Difference:  0  index  1959859
Difference:  0  index  1969940
Difference:  0  index  1994421
Difference:  0  index  2014583
Difference:  0  index  2034744
Difference:  0  index  2054905
Difference:  0  index  2075066
Difference:  0  index  2095227
Difference:  0  index  2125469
Difference:  0  index  2152830
Difference:  0  index  2165791
Difference:  0  index  2196031
Difference:  0  index  2206112
Difference:  0  index  2236352
Difference:  0  index  2256513
Difference:  0  index  2276674
Differen

In [139]:
MasterData = MasterData.drop(labels=dupliate_indices, axis=0)
MasterData = MasterData.reset_index(drop=True)

last_date=0

for i in range(len(MasterData)) :
    unix = MasterData.loc[i,'Unix Timestamp']
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            print('Difference: ', difference, ' index ',i)
    last_date=unix

In [142]:
MasterData.to_csv('Master_Data_File.csv', index=False)

In [141]:
def combine_time_frames(data, first_date, increment):
    unix_val  =0
    date_val  =''
    open_val  =0
    high_val  =-1
    low_val   =999999999
    close_val =0
    volume_val=0
    
    for x in range(increment):
        item_date = (first_date + datetime.timedelta(minutes=x))
        item_unix = item_date.replace(tzinfo=timezone.utc).timestamp()
        item_data = data.loc[data['Unix Timestamp'] == item_unix]
        volume_val+= item_data.Volume.values[0]
        if item_data.High.values[0] > high_val:
            high_val= item_data.High.values[0]
        if item_data.Low.values[0]  < low_val:
            low_val = item_data.Low.values[0]
        if x == 0: 
            open_val  = item_data.Open.values[0]
            
        if x == increment-1: 
            close_val = item_data.Close.values[0]
            date_val  = item_data.Date.values[0]
            unix_val  = item_data.values[0][0]
    
    return[unix_val,date_val,'BTCUSD',open_val,high_val,low_val,close_val,volume_val]

def make_new_time_frame(data_set, increment, time_frame_str):
    new_data_set=[]
    iterator = 0 
    for x in range(int(len(data_set)/increment)):
        first_date = datetime.datetime.utcfromtimestamp(data_set.iloc[iterator][0])
        new_row = combine_time_frames(data_set,first_date,increment)
        new_row.append(time_frame_str)
        new_data_set.append(new_row)
        iterator += increment
    return new_data_set

* 1    = 1  min 
* 5    = 5  min 
* 30   = 30 min
* 60   = 1  hour 
* 240  = 4  hour 
* 720  = 12 hour
* 1440 = 24 hour 


In [143]:
times={
    "5 Min"  : 5,
    "30 Min" : 30,
    "1 Hour" : 60,
    "4 Hour" : 240,
    "12 Hour": 720,
    "24 Hour": 1440}


for x in times: 
    combined_data = make_new_time_frame(MasterData,times[x],x)
    combined_df   = pd.DataFrame(combined_data, columns =['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume', 'Time Frame'])
    combined_df.to_csv(x+'TimeFrameData'+'.csv', index=False)
    print(x,'written to csv')


5 Min written to csv
30 Min written to csv
1 Hour written to csv
4 Hour written to csv
12 Hour written to csv
24 Hour written to csv


In [150]:
# TESTINGGGGGG
five_min_data = pd.read_csv('24 HourTimeFrameData.csv')
five_min_data
# .iloc[633620-100:633620-40]

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,Time Frame
0,1444435140,2015-10-09 23:59:00,BTCUSD,243.95,249.97,243.60,245.39,56.027841,24 Hour
1,1444521540,2015-10-10 23:59:00,BTCUSD,245.39,246.30,244.60,246.30,39.348777,24 Hour
2,1444607940,2015-10-11 23:59:00,BTCUSD,246.30,249.50,245.96,249.50,14.487783,24 Hour
3,1444694340,2015-10-12 23:59:00,BTCUSD,249.50,249.50,247.60,247.60,62.787480,24 Hour
4,1444780740,2015-10-13 23:59:00,BTCUSD,247.60,252.87,245.75,250.42,115.425756,24 Hour
...,...,...,...,...,...,...,...,...,...
2195,1634083140,2021-10-12 23:59:00,BTCUSD,57500.00,57688.88,53911.79,56019.08,1812.111067,24 Hour
2196,1634169540,2021-10-13 23:59:00,BTCUSD,56019.08,57780.80,54250.00,57382.21,1382.767620,24 Hour
2197,1634255940,2021-10-14 23:59:00,BTCUSD,57382.21,58500.02,56807.96,57319.00,1189.078324,24 Hour
2198,1634342340,2021-10-15 23:59:00,BTCUSD,57319.00,62898.00,56889.00,61665.41,3228.330492,24 Hour


In [175]:

MasterData['MA_5']  = MasterData['Close'].rolling(window=5).mean()
MasterData['MA_8']  = MasterData['Close'].rolling(window=8).mean()
MasterData['MA_10'] = MasterData['Close'].rolling(window=10).mean()
MasterData['MA_13'] = MasterData['Close'].rolling(window=13).mean()
MasterData['MA_20'] = MasterData['Close'].rolling(window=20).mean()
MasterData['MA_50'] = MasterData['Close'].rolling(window=50).mean()

delta = MasterData['Close'].diff()

up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

MasterData['RSI'] = 100 - (100/(1 + rs))

exp1 = MasterData['Close'].ewm(span=12, adjust=False).mean()
exp2 = MasterData['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
MasterData['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
MasterData['M_Signal']= exp3

MasterData.to_csv('Master_Data_File.csv', index=False)
MasterData

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,243.950,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168001,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190,60874.528,60875.47375,60881.840,60888.046154,60908.0895,60942.8306,38.784449,-23.332674,-19.401956
3168002,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391,60883.608,60881.66500,60882.704,60887.810000,60905.8600,60942.7714,51.936435,-19.013445,-19.324254
3168003,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527,60870.052,60872.53500,60873.579,60881.295385,60897.8285,60940.4124,33.238086,-24.387115,-20.336826
3168004,1634429040,2021-10-17 00:04:00,BTCUSD,60809.75,60829.33,60716.28,60716.28,0.216773,60840.358,60852.61500,60857.935,60866.667692,60885.1210,60936.6630,25.139740,-35.775636,-23.424588


In [184]:
# for getting ENv vars 
env_vars = {} # or dict {}
with open('env.txt') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        key, value = line.strip().split('=', 1)
        env_vars[key]= value
print(env_vars)

{'HOST': '64.20.33.250', 'DB': 'st19945_tickerData', 'USER': 'st19945_tickerData', 'PASSWORD': 'CU2021'}


In [185]:
# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user=env_vars['USER'],
                               pw=env_vars['PASSWORD'],
                               host=env_vars['HOST'],
                               db=env_vars['DB']))

In [187]:
################################################
# Do not run unless you are pushing the whole new data to DB 
################################################

MasterData.to_sql('BTC_Ticker_Data', con = engine, index=True, if_exists = 'append', chunksize = 1000)

pushing time frame data 

In [178]:
five_min_data = pd.read_csv('5 MinTimeFrameData.csv')

five_min_data['MA_5']  = five_min_data['Close'].rolling(window=5).mean()
five_min_data['MA_8']  = five_min_data['Close'].rolling(window=8).mean()
five_min_data['MA_10'] = five_min_data['Close'].rolling(window=10).mean()
five_min_data['MA_13'] = five_min_data['Close'].rolling(window=13).mean()
five_min_data['MA_20'] = five_min_data['Close'].rolling(window=20).mean()
five_min_data['MA_50'] = five_min_data['Close'].rolling(window=50).mean()

delta = five_min_data['Close'].diff()

up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

five_min_data['RSI'] = 100 - (100/(1 + rs))

exp1 = five_min_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = five_min_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
five_min_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
five_min_data['M_Signal']= exp3
five_min_data['Time Frame'] = five_min_data.pop('Time Frame')
five_min_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5 Min
1,1444349340,2015-10-09 00:09:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5 Min
2,1444349640,2015-10-09 00:14:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5 Min
3,1444349940,2015-10-09 00:19:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5 Min
4,1444350240,2015-10-09 00:24:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000,243.950,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,5 Min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633596,1634427840,2021-10-16 23:44:00,BTCUSD,61022.39,61022.39,60956.08,60970.43,2.655541,60978.256,60951.58250,60918.027,60904.236154,60815.0380,60848.7586,57.591414,45.920360,19.881154,5 Min
633597,1634428140,2021-10-16 23:49:00,BTCUSD,60970.43,60982.11,60913.83,60925.32,0.247314,60977.040,60954.72750,60929.182,60913.897692,60821.9010,60846.7144,54.008710,43.376702,24.580263,5 Min
633598,1634428440,2021-10-16 23:54:00,BTCUSD,60925.32,60925.32,60872.72,60872.72,0.295242,60956.390,60950.84875,60941.070,60909.888462,60834.9565,60844.3830,50.095363,36.693475,27.002906,5 Min
633599,1634428740,2021-10-16 23:59:00,BTCUSD,60872.72,60885.50,60864.75,60864.75,0.151575,60931.122,60944.25875,60937.529,60911.004615,60850.1945,60842.1874,49.509988,30.403385,27.683002,5 Min


In [188]:
five_min_data.to_sql('BTC_Ticker_Data_5_Min', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [179]:
thirty_min_data = pd.read_csv('30 MinTimeFrameData.csv')

thirty_min_data['MA_5']  = thirty_min_data['Close'].rolling(window=5).mean()
thirty_min_data['MA_8']  = thirty_min_data['Close'].rolling(window=8).mean()
thirty_min_data['MA_10'] = thirty_min_data['Close'].rolling(window=10).mean()
thirty_min_data['MA_13'] = thirty_min_data['Close'].rolling(window=13).mean()
thirty_min_data['MA_20'] = thirty_min_data['Close'].rolling(window=20).mean()
thirty_min_data['MA_50'] = thirty_min_data['Close'].rolling(window=50).mean()

delta = thirty_min_data['Close'].diff()
up    = delta.clip(lower=0)
down  = -1*delta.clip(upper=0)
ema_up   = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

thirty_min_data['RSI'] = 100 - (100/(1 + rs))

exp1 = thirty_min_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = thirty_min_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
thirty_min_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
thirty_min_data['M_Signal']= exp3

thirty_min_data['Time Frame'] = thirty_min_data.pop('Time Frame')

thirty_min_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444350540,2015-10-09 00:29:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,30 Min
1,1444352340,2015-10-09 00:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,30 Min
2,1444354140,2015-10-09 01:29:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,30 Min
3,1444355940,2015-10-09 01:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,30 Min
4,1444357740,2015-10-09 02:29:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,243.600,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,30 Min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105595,1634421540,2021-10-16 21:59:00,BTCUSD,60942.14,60960.72,60686.69,60690.92,4.974507,60828.462,60866.01625,60769.260,60769.408462,60850.7100,61323.9414,43.946016,-137.445045,-158.067370,30 Min
105596,1634423340,2021-10-16 22:29:00,BTCUSD,60690.92,60856.97,60530.80,60583.90,5.405133,60773.964,60842.66125,60809.539,60741.114615,60804.5730,61305.9856,42.140573,-148.451106,-156.144117,30 Min
105597,1634425140,2021-10-16 22:59:00,BTCUSD,60583.90,60941.21,60583.90,60813.77,4.285375,60801.976,60816.12250,60832.580,60749.449231,60809.7770,61297.8330,47.161849,-137.045129,-152.324319,30 Min
105598,1634426940,2021-10-16 23:29:00,BTCUSD,60813.77,60988.88,60723.70,60975.97,4.317270,60801.340,60814.49375,60853.103,60774.326154,60819.2585,61292.3524,50.430777,-113.608023,-144.581060,30 Min


In [189]:
thirty_min_data.to_sql('BTC_Ticker_Data_30_Min', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [180]:
one_hour_data = pd.read_csv('1 HourTimeFrameData.csv')

one_hour_data['MA_5']  = one_hour_data['Close'].rolling(window=5).mean()
one_hour_data['MA_8']  = one_hour_data['Close'].rolling(window=8).mean()
one_hour_data['MA_10'] = one_hour_data['Close'].rolling(window=10).mean()
one_hour_data['MA_13'] = one_hour_data['Close'].rolling(window=13).mean()
one_hour_data['MA_20'] = one_hour_data['Close'].rolling(window=20).mean()
one_hour_data['MA_50'] = one_hour_data['Close'].rolling(window=50).mean()

delta = one_hour_data['Close'].diff()

up   = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up   = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

one_hour_data['RSI'] = 100 - (100/(1 + rs))

exp1 = one_hour_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = one_hour_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
one_hour_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
one_hour_data['M_Signal']= exp3

one_hour_data['Time Frame'] = one_hour_data.pop('Time Frame')

one_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444352340,2015-10-09 00:59:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1 Hour
1,1444355940,2015-10-09 01:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1 Hour
2,1444359540,2015-10-09 02:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1 Hour
3,1444363140,2015-10-09 03:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1 Hour
4,1444366740,2015-10-09 04:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,243.600,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52795,1634414340,2021-10-16 19:59:00,BTCUSD,61026.08,61078.42,60843.28,60856.39,40.042673,60814.024,60843.20875,60988.874,61119.678462,61322.0410,60257.6420,49.066101,83.698289,253.361372,1 Hour
52796,1634417940,2021-10-16 20:59:00,BTCUSD,60856.39,61042.21,60410.51,60979.15,36.188282,60819.510,60876.89125,60934.730,61063.930000,61298.0510,60316.5056,50.790831,73.663753,217.421848,1 Hour
52797,1634421540,2021-10-16 21:59:00,BTCUSD,60979.15,61294.68,60686.69,60690.92,34.234604,60827.180,60835.51000,60841.574,60998.403846,61262.7290,60372.8174,46.785040,41.969757,182.331430,1 Hour
52798,1634425140,2021-10-16 22:59:00,BTCUSD,60690.92,60941.21,60530.80,60813.77,9.690508,60873.262,60819.24500,60851.982,60951.736923,61220.6275,60440.0124,48.644185,26.459991,151.157142,1 Hour


In [190]:
one_hour_data.to_sql('BTC_Ticker_Data_1_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [181]:
four_hour_data = pd.read_csv('4 HourTimeFrameData.csv')

four_hour_data['MA_5']  = four_hour_data['Close'].rolling(window=5).mean()
four_hour_data['MA_8']  = four_hour_data['Close'].rolling(window=8).mean()
four_hour_data['MA_10'] = four_hour_data['Close'].rolling(window=10).mean()
four_hour_data['MA_13'] = four_hour_data['Close'].rolling(window=13).mean()
four_hour_data['MA_20'] = four_hour_data['Close'].rolling(window=20).mean()
four_hour_data['MA_50'] = four_hour_data['Close'].rolling(window=50).mean()

delta = four_hour_data['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

four_hour_data['RSI'] = 100 - (100/(1 + rs))

exp1 = four_hour_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = four_hour_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
four_hour_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
four_hour_data['M_Signal']= exp3

four_hour_data['Time Frame'] = four_hour_data.pop('Time Frame')

four_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444363140,2015-10-09 03:59:00,BTCUSD,243.95,243.95,243.60,243.60,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,4 Hour
1,1444377540,2015-10-09 07:59:00,BTCUSD,243.60,243.60,243.60,243.60,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,4 Hour
2,1444391940,2015-10-09 11:59:00,BTCUSD,243.60,243.75,243.60,243.63,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.002393,0.000479,4 Hour
3,1444406340,2015-10-09 15:59:00,BTCUSD,243.63,244.00,243.63,244.00,8.145083,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.033757,0.007134,4 Hour
4,1444420740,2015-10-09 19:59:00,BTCUSD,244.00,244.06,243.70,244.06,3.662625,243.778,NaN,NaN,NaN,NaN,NaN,100.000000,0.062731,0.018253,4 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13195,1634371140,2021-10-16 07:59:00,BTCUSD,61624.84,62366.08,61555.95,61703.88,129.040234,61292.586,60583.40625,59986.158,59400.152308,58363.3235,56714.5684,72.806379,1473.424472,1127.472323,4 Hour
13196,1634385540,2021-10-16 11:59:00,BTCUSD,61703.88,62028.03,61350.01,61622.48,158.043904,61613.702,60860.27250,60360.873,59726.789231,58629.9945,56870.4842,72.001724,1498.668789,1201.711617,4 Hour
13197,1634399940,2021-10-16 15:59:00,BTCUSD,61622.48,61622.48,60468.00,60951.72,211.221619,61513.666,61074.23000,60724.145,59982.844615,58945.0995,56992.6058,65.570735,1447.860334,1250.941360,4 Hour
13198,1634414340,2021-10-16 19:59:00,BTCUSD,60951.72,61205.99,60174.14,60856.39,188.134971,61351.862,61236.69000,60869.029,60253.243846,59227.1265,57104.1196,64.686444,1383.948634,1277.542815,4 Hour


In [191]:
four_hour_data.to_sql('BTC_Ticker_Data_4_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [182]:
twelve_hour_data = pd.read_csv('12 HourTimeFrameData.csv')

twelve_hour_data['MA_5']  = twelve_hour_data['Close'].rolling(window=5).mean()
twelve_hour_data['MA_8']  = twelve_hour_data['Close'].rolling(window=8).mean()
twelve_hour_data['MA_10'] = twelve_hour_data['Close'].rolling(window=10).mean()
twelve_hour_data['MA_13'] = twelve_hour_data['Close'].rolling(window=13).mean()
twelve_hour_data['MA_20'] = twelve_hour_data['Close'].rolling(window=20).mean()
twelve_hour_data['MA_50'] = twelve_hour_data['Close'].rolling(window=50).mean()

delta = twelve_hour_data['Close'].diff()

up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

twelve_hour_data['RSI'] = 100 - (100/(1 + rs))

exp1 = twelve_hour_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = twelve_hour_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
twelve_hour_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
twelve_hour_data['M_Signal']= exp3

twelve_hour_data['Time Frame'] = twelve_hour_data.pop('Time Frame')


twelve_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444391940,2015-10-09 11:59:00,BTCUSD,243.95,243.95,243.60,243.63,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,12 Hour
1,1444435140,2015-10-09 23:59:00,BTCUSD,243.63,249.97,243.63,245.39,52.027841,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.140399,0.028080,12 Hour
2,1444478340,2015-10-10 11:59:00,BTCUSD,245.39,245.54,244.60,244.60,27.223703,NaN,NaN,NaN,NaN,NaN,NaN,96.662442,0.185778,0.059619,12 Hour
3,1444521540,2015-10-10 23:59:00,BTCUSD,244.60,246.30,244.60,246.30,12.125075,NaN,NaN,NaN,NaN,NaN,NaN,96.902054,0.354827,0.118661,12 Hour
4,1444564740,2015-10-11 11:59:00,BTCUSD,246.30,246.30,246.30,246.30,0.080999,245.244,NaN,NaN,NaN,NaN,NaN,96.902054,0.483229,0.191575,12 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,1634255940,2021-10-14 23:59:00,BTCUSD,57623.00,58090.65,56807.96,57319.00,654.101934,56711.828,56846.59125,56464.079,56028.610769,54994.3315,48495.0614,66.130286,2430.820878,2519.209279,12 Hour
4396,1634299140,2021-10-15 11:59:00,BTCUSD,57319.00,60000.00,56889.00,59556.71,744.116808,57419.354,57236.44125,56903.418,56459.942308,55476.5255,48811.3752,71.404430,2535.530115,2522.473446,12 Hour
4397,1634342340,2021-10-15 23:59:00,BTCUSD,59556.71,62898.00,59215.31,61665.41,2484.213684,58709.266,57757.11750,57599.485,56987.030000,55983.5230,49184.5660,75.306684,2756.887721,2569.356301,12 Hour
4398,1634385540,2021-10-16 11:59:00,BTCUSD,61665.41,62366.08,61129.04,61622.48,438.366647,59557.320,58300.46750,58117.942,57498.876154,56497.2690,49550.9806,75.082046,2895.473905,2634.579822,12 Hour


In [192]:
twelve_hour_data.to_sql('BTC_Ticker_Data_12_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)

In [183]:
twenty_four_hour_data = pd.read_csv('24 HourTimeFrameData.csv')

twenty_four_hour_data['MA_5']  = twenty_four_hour_data['Close'].rolling(window=5).mean()
twenty_four_hour_data['MA_8']  = twenty_four_hour_data['Close'].rolling(window=8).mean()
twenty_four_hour_data['MA_10'] = twenty_four_hour_data['Close'].rolling(window=10).mean()
twenty_four_hour_data['MA_13'] = twenty_four_hour_data['Close'].rolling(window=13).mean()
twenty_four_hour_data['MA_20'] = twenty_four_hour_data['Close'].rolling(window=20).mean()
twenty_four_hour_data['MA_50'] = twenty_four_hour_data['Close'].rolling(window=50).mean()

delta = twenty_four_hour_data['Close'].diff()

up   = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up   = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down

twenty_four_hour_data['RSI'] = 100 - (100/(1 + rs))

exp1 = twenty_four_hour_data['Close'].ewm(span=12, adjust=False).mean()
exp2 = twenty_four_hour_data['Close'].ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
twenty_four_hour_data['MACD']= macd
exp3 = macd.ewm(span=9, adjust=False).mean()
twenty_four_hour_data['M_Signal']= exp3

twenty_four_hour_data['Time Frame'] = twenty_four_hour_data.pop('Time Frame')


twenty_four_hour_data

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume,MA_5,MA_8,MA_10,MA_13,MA_20,MA_50,RSI,MACD,M_Signal,Time Frame
0,1444435140,2015-10-09 23:59:00,BTCUSD,243.95,249.97,243.60,245.39,56.027841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,24 Hour
1,1444521540,2015-10-10 23:59:00,BTCUSD,245.39,246.30,244.60,246.30,39.348777,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.072593,0.014519,24 Hour
2,1444607940,2015-10-11 23:59:00,BTCUSD,246.30,249.50,245.96,249.50,14.487783,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.383911,0.088397,24 Hour
3,1444694340,2015-10-12 23:59:00,BTCUSD,249.50,249.50,247.60,247.60,62.787480,NaN,NaN,NaN,NaN,NaN,NaN,88.017478,0.471879,0.165093,24 Hour
4,1444780740,2015-10-13 23:59:00,BTCUSD,247.60,252.87,245.75,250.42,115.425756,247.842,NaN,NaN,NaN,NaN,NaN,89.943552,0.760379,0.284150,24 Hour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,1634083140,2021-10-12 23:59:00,BTCUSD,57500.00,57688.88,53911.79,56019.08,1812.111067,55428.340,54722.92125,53526.201,51918.111538,48667.0205,47881.4790,66.167962,2834.092876,1817.931058,24 Hour
2196,1634169540,2021-10-13 23:59:00,BTCUSD,56019.08,57780.80,54250.00,57382.21,1382.767620,56114.902,55455.01500,54440.196,52960.583077,49291.2715,48074.8660,68.397181,2995.709610,2053.486769,24 Hour
2197,1634255940,2021-10-14 23:59:00,BTCUSD,57382.21,58500.02,56807.96,57319.00,1189.078324,56585.006,55704.27875,55248.458,53664.632308,50014.8245,48241.0330,68.172860,3083.150874,2259.419590,24 Hour
2198,1634342340,2021-10-15 23:59:00,BTCUSD,57319.00,62898.00,56889.00,61665.41,3228.330492,57977.140,56688.54000,56262.453,54740.663846,50962.5540,48536.7832,74.392069,3463.245529,2500.184777,24 Hour


In [193]:
twenty_four_hour_data.to_sql('BTC_Ticker_Data_24_Hour', con = engine,index=True, if_exists = 'append', chunksize = 1000)


Testing Below this line 
***************


In [131]:
testing_set = MasterData
testing_set = testing_set.drop(labels=dupliate_indices, axis=0)


In [137]:
print(len(MasterData))
print(len(testing_set))
testing_set = testing_set.reset_index(drop=True)

last_date=0
dupliate_indices=[]
for i in range(len(testing_set)) :
    unix = testing_set.loc[i,'Unix Timestamp']
#     date = datetime.datetime.utcfromtimestamp()
    if i !=0:
        difference = unix-last_date
        if (difference!=60):
            dupliate_indices.append(i)
            print('Difference: ', difference, ' index ',i)
    last_date=unix

testing_set

3168107
3168006


,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1444348800,2015-10-09 00:00:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
1,1444348860,2015-10-09 00:01:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
2,1444348920,2015-10-09 00:02:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
3,1444348980,2015-10-09 00:03:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
4,1444349040,2015-10-09 00:04:00,BTCUSD,243.95,243.95,243.95,243.95,0.000000
...,...,...,...,...,...,...,...,...
3168001,1634428860,2021-10-17 00:01:00,BTCUSD,60877.53,60877.53,60851.85,60875.98,0.182190
3168002,1634428920,2021-10-17 00:02:00,BTCUSD,60875.98,60937.30,60854.84,60922.25,1.135391
3168003,1634428980,2021-10-17 00:03:00,BTCUSD,60922.25,60925.99,60809.75,60809.75,0.108527
3168004,1634429040,2021-10-17 00:04:00,BTCUSD,60809.75,60829.33,60716.28,60716.28,0.216773


In [ ]:
# first_date = datetime.datetime.utcfromtimestamp(temp_set.iloc[0][0])
# combine_time_frames(temp_set,first_date,30)


In [ ]:
# five_min_data = make_new_time_frame(temp_set,5,time_frame_str)
# for y in five_min_data:
#     print(y)

# five_min_df = pd.DataFrame(five_min_data, columns =['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume', time_frame_str])
# five_min_df